# Sentiment analysis of Amazon product reviews using logistic regression

### Fire up GraphLab Create

In [129]:
from __future__ import division
import graphlab
import math
import string

## Data preparation

### Load the product review data
We will use the dataset consisting of baby product reviews on Amazon.com.

In [130]:
products = graphlab.SFrame('amazon_baby.gl/')

### Explore the data

In [131]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


### Build the word count vector for each review

Let us explore a specific example of a baby product.


In [132]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

### Transform the data

Remove punctuation and transform the reviews into word-counts.

In [133]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [134]:
products[269]['word_count']

{'a': 1L, 'favorite': 1L, 'house': 1L, 'in': 1L, 'our': 1L}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [135]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [136]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 3L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3L, 'ingenious':1L, 'love': 2L, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'all': 2L,'help': 1L, 'cried': 1L, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'habit': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'bound': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2L, 'all': 1L,'right': 1L, 'able': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'fantastic':1L, 'help': 1L, 'give': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1L, 'standarad':1L, 'another': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2L, 'nannys': 1L,'just': 1L, 'sleep': 2L, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

### Split the data

In [137]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


## Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target.

In [138]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 3.917418     | 0.960964          |

| 6         | 13       | 3.000000  | 4.288407     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [139]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 4.895

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]       : 41.9847
word_count[placeid]            : 41.7354
word_count[labelbox]           : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
word_count[knobskeeping]       : 36.2091

Lowest Negative Coeffic

Now that we have fitted the model, we can extract the weights (coefficients) and have a look.

In [140]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model.

In [141]:
weights[1]

{'class': 1L,
 'index': 'recommend',
 'name': 'word_count',
 'stderr': None,
 'value': 0.30381560001529373}

Lets calculate how many coefficients (weights) corresponding to particular word are positive or negative

In [142]:
num_positive_weights = (weights['value'] >=0).sum()
num_negative_weights = (weights['value'] < 0).sum()

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 68419 
Number of negative weights: 53294 


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.

In [143]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2L, 'all': 1L,'love': 1L, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1L, 'wall': 1L,'them': 1L, 'decals': ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1L, 'money': 1L,'into': 1L, 'it': 3L, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [144]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like.

In [145]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

Prediction using the `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. The **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.

For each row, the **score** (or margin) is a number in the range **[-infinity, infinity]**.

In [146]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[6.734619727059076, -5.734130996760209, -14.668460404468478]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, function to calculate $\hat{y}$, the class predictions:

In [147]:
def class_predictions(scores):
    preds = []
    for score in scores:
        if score > 0:
            pred = 1
        else:
            pred = -1
        preds.append(pred)
    return preds

Code to verify that the class predictions obtained by our calculations are the same as that obtained from GraphLab Create.

In [148]:
print "Class predictions using our calculation: " ,class_predictions(scores)
print "Class predictions according to GraphLab Create: " ,sentiment_model.predict(sample_test_data)

Class predictions using our calculation:  [1, -1, -1]
Class predictions according to GraphLab Create:  [1L, -1L, -1L]


## Probability predictions

We can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [149]:
def probability_predictions(scores):
    probabilities = []
    for score in scores:
        probability = 1 / (1 + math.exp(-score))
        probabilities.append(probability)
    return(probabilities)

Making sure that out probability predictions match the ones obtained from GraphLab Create.

In [150]:
print "Probability predictions using our calculation: " ,probability_predictions(scores)
print "Class predictions according to GraphLab Create:" ,sentiment_model.predict(sample_test_data, output_type='probability')

Probability predictions using our calculation:  [0.9988123848377196, 0.0032232681818008375, 4.2615579966555943e-07]
Class predictions according to GraphLab Create: [0.9988123848377195, 0.003223268181800837, 4.2615579966555916e-07]


## Find the most positive (and negative) review

Using the `sentiment_model`, we find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**.

In [151]:
test_data['probability'] = sentiment_model.predict(test_data, output_type='probability')
test_data

name,review,rating,word_count,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1L, 'standarad':1L, 'another': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2L, 'nannys': 1L,'just': 1L, 'sleep': 2L, ...",1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5.0,"{'and': 1L, 'babys': 1L,'love': 1L, 'like': 1L, ...",1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5.0,"{'and': 3L, 'all': 1L,'months': 1L, ...",1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,"{'and': 2L, 'because':1L, 'family': 1L, ...",1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5.0,"{'all': 1L, 'being': 1L,'infant': 1L, 'course': ...",1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5.0,"{'and': 1L, 'own': 1L,'it': 3L, 'our': 1L, ...",1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2.0,"{'and': 3L, 'cute': 1L,'rating': 1L, ...",-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5.0,"{'beautiful': 1L, 'and':2L, 'love': 1L, 'elmo': ...",1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5.0,"{'remembering': 1L,'and': 4L, ...",1


### Top 20 Most positive reviews

In [154]:
test_data['name', 'probability'].topk('probability', k=20).print_rows(20)

+-------------------------------+-------------+
|              name             | probability |
+-------------------------------+-------------+
| Britax Decathlon Convertib... |     1.0     |
| Ameda Purely Yours Breast ... |     1.0     |
| Traveling Toddler Car Seat... |     1.0     |
| Shermag Glider Rocker Comb... |     1.0     |
| Cloud b Sound Machine Soot... |     1.0     |
| JP Lizzy Chocolate Ice Cla... |     1.0     |
| Fisher-Price Rainforest Me... |     1.0     |
| Lilly Gold Sit 'n' Stroll ... |     1.0     |
|  Fisher-Price Deluxe Jumperoo |     1.0     |
| North States Supergate Pre... |     1.0     |
|   Munchkin Mozart Magic Cube  |     1.0     |
| Britax Marathon Convertibl... |     1.0     |
| Wizard Convertible Car Sea... |     1.0     |
|   Capri Stroller - Red Tech   |     1.0     |
| Peg Perego Primo Viaggio C... |     1.0     |
| HALO SleepSack Micro-Fleec... |     1.0     |
| Leachco Snoogle Total Body... |     1.0     |
| Summer Infant Complete Nur... |     1.

### Top 20 Most negative reviews

In [155]:
test_data['name','probability'].topk('probability', k=20, reverse=True).print_rows(20)

+-------------------------------+--------------------+
|              name             |    probability     |
+-------------------------------+--------------------+
| Jolly Jumper Arctic Sneak ... | 7.80415068208e-100 |
| Levana Safe N'See Digital ... | 6.83650885514e-25  |
| Snuza Portable Baby Moveme... | 2.12654510823e-24  |
| Fisher-Price Ocean Wonders... | 2.24582080779e-23  |
| VTech Communications Safe ... | 1.32962966148e-22  |
| Safety 1st High-Def Digita... | 2.06872097469e-20  |
| Chicco Cortina KeyFit 30 T... | 5.93881994672e-20  |
| Prince Lionheart Warmies W... | 6.28510016539e-20  |
| Valco Baby Tri-mode Twin S... | 8.05528712691e-20  |
| Adiri BPA Free Natural Nur... | 8.46521724944e-20  |
| Munchkin Nursery Projector... |  1.5285394517e-19  |
| The First Years True Choic... | 1.77901889379e-19  |
| Nuby Natural Touch Silicon... | 1.15227353848e-18  |
| Peg-Perego Tatamia High Ch... | 1.26175666136e-18  |
|    Fisher-Price Royal Potty   | 1.60282966315e-18  |
| Safety 1

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier.


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

In [156]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    predicted_labels = model.predict(data)
    
    # Compute the number of correctly classified examples
    correctly_classified_labels = sum(predicted_labels == true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = correctly_classified_labels / len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [157]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [158]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [159]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above for both train and test dataset.

In [160]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [161]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [162]:
print train_data[0]['word_count']

{'and': 3L, 'love': 1L, 'it': 3L, 'highly': 1L, 'osocozy': 1L, 'bags': 1L, 'leak': 1L, 'moist': 1L, 'does': 1L, 'recommend': 1L, 'was': 1L, 'wipes': 1L, 'disappointed': 1L, 'early': 1L, 'not': 2L, 'now': 1L, 'holder': 1L, 'wipe': 1L, 'keps': 1L, 'wise': 1L, 'i': 1L, 'planet': 1L, 'my': 2L, 'came': 1L}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [163]:
print train_data[0]['word_count_subset']

{'love': 1L, 'disappointed': 1L}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [164]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.554474     | 0.866815          |

| 6         | 7        | 0.646720     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.6728

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [165]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [168]:
simple_weights = simple_model.coefficients
simple_weights

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,work,1,-0.621700012425,0.0230330597946
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,great,1,0.94469126948,0.0209509926591


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [169]:
simple_weights.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

**Postive coefficients in the simple model**

In [172]:
sum(simple_model.coefficients['value'] >= 0) - 1

10L

In [173]:
positive_significant_weights = simple_model.coefficients[(simple_model.coefficients['value'] >= 0)
                                                         & (simple_model.coefficients['name'] != '(intercept)')]
positive_significant_words = positive_significant_weights['index']
print len(positive_significant_words)
print positive_significant_words

10
['love', 'well', 'loves', 'little', 'easy', 'great', 'able', 'perfect', 'old', 'car']


**Positive coefficients (words) of the simple model are also positive in the sentiment model. Let's see**

In [174]:
sentiment_model.coefficients.filter_by(positive_significant_words, 'index')

name,index,class,value,stderr
word_count,love,1,1.43301685439,None
word_count,well,1,0.627964877567,None
word_count,loves,1,1.5664851757,None
word_count,little,1,0.674162457499,None
word_count,easy,1,1.21346937822,None
word_count,great,1,1.31459245039,None
word_count,able,1,0.174331272552,None
word_count,perfect,1,1.75190114392,None
word_count,old,1,0.00912230113671,None
word_count,car,1,0.195263670618,None


## Comparing models

Classification accuracy of the **sentiment_model** on the **train_data**:

In [175]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.979440247046831

Classification accuracy of the **simple_model** on the **train_data**:

In [176]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

**Hence classification accuracy of sentiment model is better than simple model on train data.**

Classification accuracy of the **sentiment_model** on the **test_data**:

In [177]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

Classification accuracy of the **simple_model** on the **test_data**:

In [178]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

**Hence classification accuracy of sentiment model is better than simple model on test data too.**

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, one should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

Majority class in the train_data

In [180]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Accuracy of the majority class classifier on **test_data**.

In [181]:
print (test_data['sentiment'] == +1).sum()
print (test_data['sentiment'] == -1).sum()

28095
5241


In [182]:
print (test_data['sentiment'] == +1).sum()/len(test_data['sentiment'])

0.842782577394


### **Hence classification accuracy of sentiment model is far better than simple model or the majority class classifier.**